# Analysis of the NYC flights dataset for 2013

In this notebook notebook, we're going to explore the NYC flights dataset and see if we can find interesting patterns.

<img src='Newark.jpg'>

In [139]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [3]:
# Loading the data
df = pd.read_csv('flights.csv')

In [5]:
# Get the dimensions of the dataset
df.shape

(336776, 19)

In [6]:
# Column names
df.columns

Index(['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour'],
      dtype='object')

In [7]:
# Datatypes of columns
df.dtypes

year                int64
month               int64
day                 int64
dep_time          float64
sched_dep_time      int64
dep_delay         float64
arr_time          float64
sched_arr_time      int64
arr_delay         float64
carrier            object
flight              int64
tailnum            object
origin             object
dest               object
air_time          float64
distance            int64
hour                int64
minute              int64
time_hour          object
dtype: object

In [8]:
# Preview of what we're working with
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


In [27]:
# The percentage of non null data for each column
df.describe().loc['count'] / df.shape[0]

year              1.000000
month             1.000000
day               1.000000
dep_time          0.975488
sched_dep_time    1.000000
dep_delay         0.975488
arr_time          0.974128
sched_arr_time    1.000000
arr_delay         0.971999
flight            1.000000
air_time          0.971999
distance          1.000000
hour              1.000000
minute            1.000000
Name: count, dtype: float64

In [29]:
# The percentage of data that's left when considering only complete rows
df.dropna().describe().loc['count'] / df.shape[0]

year              0.971999
month             0.971999
day               0.971999
dep_time          0.971999
sched_dep_time    0.971999
dep_delay         0.971999
arr_time          0.971999
sched_arr_time    0.971999
arr_delay         0.971999
flight            0.971999
air_time          0.971999
distance          0.971999
hour              0.971999
minute            0.971999
Name: count, dtype: float64

In [36]:
# Exploring rows that have a null field
mask = df.isnull().any(axis=1)
df[mask]

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
471,2013,1,1,1525.0,1530,-5.0,1934.0,1805,NaN,MQ,4525,N719MQ,LGA,XNA,NaN,1147,15,30,2013-01-01 15:00:00
477,2013,1,1,1528.0,1459,29.0,2002.0,1647,NaN,EV,3806,N17108,EWR,STL,NaN,872,14,59,2013-01-01 14:00:00
615,2013,1,1,1740.0,1745,-5.0,2158.0,2020,NaN,MQ,4413,N739MQ,LGA,XNA,NaN,1147,17,45,2013-01-01 17:00:00
643,2013,1,1,1807.0,1738,29.0,2251.0,2103,NaN,UA,1228,N31412,EWR,SAN,NaN,2425,17,38,2013-01-01 17:00:00
725,2013,1,1,1939.0,1840,59.0,29.0,2151,NaN,9E,3325,N905XJ,JFK,DFW,NaN,1391,18,40,2013-01-01 18:00:00
733,2013,1,1,1952.0,1930,22.0,2358.0,2207,NaN,EV,4333,N11194,EWR,TUL,NaN,1215,19,30,2013-01-01 19:00:00
754,2013,1,1,2016.0,1930,46.0,NaN,2220,NaN,EV,4204,N14168,EWR,OKC,NaN,1325,19,30,2013-01-01 19:00:00
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,2013-01-01 16:00:00
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,2013-01-01 19:00:00
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,2013-01-01 15:00:00


In [53]:
# Keeping only complete records
df_clean = df.dropna()

In [58]:
# Finding out the busiest month
df_clean.groupby('month')['flight'].count().sort_values(ascending=False)

month
8     28756
10    28618
7     28293
5     28128
3     27902
4     27564
6     27075
12    27020
9     27010
11    26971
1     26398
2     23611
Name: flight, dtype: int64

In [90]:
# Finding out the most frequent flight
busiest_flight = df_clean.groupby('flight')['day'].count().idxmax()
df_clean.loc[df_clean['flight'] == busiest_flight, ['carrier', 'origin', 'dest']].drop_duplicates()

,carrier,origin,dest
379,UA,EWR,HNL
590,B6,JFK,FLL
27109,AS,EWR,SEA
27232,B6,JFK,SFO
224929,US,JFK,PHX


In [96]:
# Finding out the most frequent flight (take 2)
df_clean.assign(org_dep=df_clean['origin']+'_'+df_clean['dest']).groupby('org_dep')['day'].count().idxmax()

'JFK_LAX'

In [144]:
def convert_dep_time(row, dep):
    date = '%s%s%s' % (str(row['year']), '%02d' % row['month'], '%02d' % row['day'])
    time = '%04d' % row[dep] if row[dep] < 2400 else '0000'
    return datetime.datetime.strptime(date + time, '%Y%m%d%H%M')

def convert_arr_time(row, arr):
    date = '%s%s%s' % (str(row['year']), '%02d' % row['month'], '%02d' % row['day'])
    time = '%04d' % row[arr] if row[arr] < 2400 else '0000'
    if row['dep_time'] < row['arr_time']:
        return datetime.datetime.strptime(date + time, '%Y%m%d%H%M')
    else:
        return datetime.datetime.strptime(date + time, '%Y%m%d%H%M') + datetime.timedelta(days=1)
        

In [145]:
df_clean.apply(convert_arr_time, arr='arr_time', axis=1)

0        2013-01-01 08:30:00
1        2013-01-01 08:50:00
2        2013-01-01 09:23:00
3        2013-01-01 10:04:00
4        2013-01-01 08:12:00
5        2013-01-01 07:40:00
6        2013-01-01 09:13:00
7        2013-01-01 07:09:00
8        2013-01-01 08:38:00
9        2013-01-01 07:53:00
10       2013-01-01 08:49:00
11       2013-01-01 08:53:00
12       2013-01-01 09:24:00
13       2013-01-01 09:23:00
14       2013-01-01 09:41:00
15       2013-01-01 07:02:00
16       2013-01-01 08:54:00
17       2013-01-01 08:51:00
18       2013-01-01 08:37:00
19       2013-01-01 08:44:00
20       2013-01-01 08:12:00
21       2013-01-01 08:21:00
22       2013-01-01 08:58:00
23       2013-01-01 08:37:00
24       2013-01-01 08:58:00
25       2013-01-01 08:07:00
26       2013-01-01 09:45:00
27       2013-01-01 09:25:00
28       2013-01-01 10:39:00
29       2013-01-01 08:33:00
                 ...        
336740   2013-09-30 22:46:00
336741   2013-09-30 23:17:00
336742   2013-09-30 23:10:00
336743   2013-